### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [2]:
# Do the data count to see if there are missing value in each rows / columns 
#purchase_data.count()

In [3]:
# See the data columns and the first 3 rows of our data 
purchase_data.head(3)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88


## Player Count

* Display the total number of players


In [4]:
#Calculate the number of player in the DataFrame and display it as dataframe
#find the number of player and not counting duplicate using the .value_count() function
unique_player = len(purchase_data["SN"].value_counts())

#Display it as a new table 
table1 = pd.DataFrame({"Total Players" : [unique_player]})
table1

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [5]:
unique_items = len(purchase_data["Item ID"].unique())
price_avg = purchase_data["Price"].mean()
number_purchases = purchase_data["Purchase ID"].count()
total_rev = purchase_data["Price"].sum()

summary_df_table = pd.DataFrame({"Number of Unique Items" : [unique_items],
                                "Average Price" : [price_avg], 
                                 "Number of Purchases" : [number_purchases],
                                 "Total Revenue" : [total_rev]})

#use dictionary to format specific columns = "pandas.pydata.org/pandas-docs/stable/user_guide/style.html" 
summary_df_table.style.format({'Average Price': "${:,.2f}", "Total Revenue":"${:,}"})


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [6]:
#using .nunique() to perform my data filter so I dont have to recreate a new pd.DataFrame table everytime i finish my filter
#I only need to join columns using concat build in function to by filter data
#Rename my columns to match the required outcome on the assignment

#find exact gender count with no duplicates - .nunique() - geeksforgeeks examples
no_duplicate = purchase_data.groupby("Gender")["SN"].nunique()

#find the percent value for each gender by dividing the # of player in our data without duplicates
percent = no_duplicate / unique_player * 100

#Join the 2 series - pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
#https://www.youtube.com/watch?v=wiw9Qo08TIk - undertood how it turns to a Dataframe object
#make sure to use axis=1 to display it as a columns 
result = pd.concat([no_duplicate, percent], axis=1, join='inner')

#Use "result.columns" - to view the columns name
#https://stackoverflow.com/questions/28503445/assigning-column-names-to-a-pandas-series
#use stackoverflow to rename my column
result.columns = ["Total Count", "Percent of Players"]

#reset index to none 
result.index.name = None

#format the Percent of Players column to show appropiate % with 2 decimal value 
result["Percent of Players"] = result["Percent of Players"].map("{:.2f}%".format) 
result.sort_values(by = ['Total Count'], ascending=False)

,Total Count,Percent of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
#using nunique() to keep my dataframe intact and join all the new result columns

total_gender_purchase = purchase_data.groupby("Gender")["Purchase ID"].nunique()
avg_purchase_price = purchase_data.groupby("Gender")["Price"].mean()
avg_purchase_total = purchase_data.groupby("Gender")["Price"].sum()
avg_total_purchase_per_person = avg_purchase_total / no_duplicate

#combine all of the individual panda series into a dataframe object and index will be set at Gender
purchasing_analysis = pd.concat([total_gender_purchase, avg_purchase_price,
                    avg_purchase_total, avg_total_purchase_per_person]
                   , axis=1, join='inner')

#Use "purchasing_analysis.columns" - to find out all columns names in our data so far and to rename it

#rename columns name 
purchasing_analysis.columns = ["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Avg Total Purchase per Person"]


#format columns' value to display correct monetary 
purchasing_analysis["Average Purchase Price"] = purchasing_analysis["Average Purchase Price"].map("${:.2f}".format)
purchasing_analysis["Total Purchase Value"] = purchasing_analysis["Total Purchase Value"].map("${:1,.2f}".format)
purchasing_analysis["Avg Total Purchase per Person"] = purchasing_analysis["Avg Total Purchase per Person"].map("${:.2f}".format) 
purchasing_analysis


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [8]:
# Age Demographics - using max and min function to find our range 
maxi = purchase_data["Age"].max()
mini = purchase_data["Age"].min()
print(maxi, mini)

45 7


In [9]:
#Create bins in which data will be group into
bins = [0, 9, 14, 19, 24, 29, 34, 39, 50]
group_range = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "30-39", "40+"]

#Add column of bins based on Age
purchase_data["Age Ranges"] = pd.cut(purchase_data["Age"], bins=bins, labels=group_range)

#group it by "Age Range" column
age_group = purchase_data.groupby("Age Ranges")["SN"].nunique()

#find percent value for each "Age Range" column - divide by number of purchaser without duplicate
percent_of_players = age_group / unique_player * 100

age_demographic = pd.concat([age_group, percent_of_players], axis=1, join='inner')

#Use "age_demographic.columns" - to find out all columns names in our data so far and to rename it
age_demographic.columns = ["Total Count", "Percentage of Players"]

age_demographic.index.name = None
age_demographic["Percentage of Players"] = age_demographic["Percentage of Players"].map("{:,.2f}%".format) 

age_demographic

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
30-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [10]:
#Calculate total purchase count, avg. purchase price, and avg purchase total per person
#ag = age group 


purchase_count_by_ag = purchase_data.groupby("Age Ranges")["Purchase ID"].count()
avg_purchase_by_ag = purchase_data.groupby("Age Ranges")["Price"].mean()
total_purchase_value_by_ag = purchase_data.groupby("Age Ranges")["Price"].sum()
avg_total_individual_purchase = total_purchase_value_by_ag / purchase_count_by_ag

#Use concat to combine all of our dataframe series 
purchasing_analysis_by_ag = pd.concat([purchase_count_by_ag, avg_purchase_by_ag,
                                    total_purchase_value_by_ag, avg_total_individual_purchase], axis=1, join='inner')

#Use "purchasing_analysis_by_ag.columns" to check our columns name to be rename
purchasing_analysis_by_ag.columns

#rename columns
purchasing_analysis_by_ag.columns =["Purchase Count", "Average Purchase Price",
                                   "Total Purchase Value", "Avg Total Purchase per Person"]

purchasing_analysis_by_ag["Average Purchase Price"] = purchasing_analysis_by_ag["Average Purchase Price"].map("${:,.2f}".format)
purchasing_analysis_by_ag["Total Purchase Value"] = purchasing_analysis_by_ag["Total Purchase Value"].map("${:1,.2f}".format)
purchasing_analysis_by_ag["Avg Total Purchase per Person"] = purchasing_analysis_by_ag["Avg Total Purchase per Person"].map("${:,.2f}".format)


purchasing_analysis_by_ag

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$3.35
10-14,28,$2.96,$82.78,$2.96
15-19,136,$3.04,$412.89,$3.04
20-24,365,$3.05,"$1,114.06",$3.05
25-29,101,$2.90,$293.00,$2.90
30-34,73,$2.93,$214.00,$2.93
30-39,41,$3.60,$147.67,$3.60
40+,13,$2.94,$38.24,$2.94


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [11]:
#Group our data using "SN" column to find who is our top buyers
#find the average purchase price and Total purchase value 
purchase_count = purchase_data.groupby("SN")["Purchase ID"].nunique()

individual_purchaser_avg = purchase_data.groupby("SN")["Price"].mean()

total_purchase_value = purchase_data.groupby("SN")["Price"].sum()

#Use concat to combines all of our columns
top_spenders = pd.concat([purchase_count, individual_purchaser_avg,
                    total_purchase_value]
                   , axis=1, join='inner')

#Use "top_spenders.columns" to find our columns title

#rename columns' name 
top_spenders.columns = ["Purchase Count", "Average Purchase Price", "Total Purchase Value"]

#reformat monetary values to reflect in our data. 
top_spenders["Average Purchase Price"] = top_spenders["Average Purchase Price"].map("${:.2f}".format)
top_spenders["Total Purchase Value"] = top_spenders["Total Purchase Value"].map("${:.2f}".format)

#Display the first 5 top spender
top_spenders.sort_values(by = ["Purchase Count"], ascending=False).head(5)


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Iral74,4,$3.40,$13.62
Idastidru52,4,$3.86,$15.45
Asur53,3,$2.48,$7.44
Inguron55,3,$3.70,$11.11


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [12]:
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Ranges
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24


In [13]:
#Retrive 3 columns - Item ID, Item Name, and Item Price
retrieved_item = purchase_data[["Item ID", "Item Name", "Price"]]

#Group item ID & item Name 
group_item = retrieved_item.groupby(["Item ID", "Item Name"]).count()


#top_spenders = pd.concat([group_item, purchase_count], axis=1, join='inner')
#top_spenders.columns = ["Item ID", "Item Name", "Purchase Count", "Item Price", "Total Purchase Value"]

purchase_count = group_item.sort_values(by = "Price", ascending=False)
purchase_count

,,Price
Item ID,Item Name,
178,"Oathbreaker, Last Hope of the Breaking Storm",12
145,Fiery Glass Crusader,9
108,"Extraction, Quickblade Of Trembling Hands",9
82,Nirvana,9
19,"Pursuit, Cudgel of Necromancy",8
...,...,...
104,Gladiator's Glaive,1
23,Crucifer,1
180,Stormcaller,1


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame

